# Accelerating End-to-End Data Science Workflows # 


## 05 - การจัดกลุ่ม (Grouping) ##

**สารบัญ**

โน้ตบุ๊กนี้จะอภิปรายและสาธิตวิธีการใช้การจัดกลุ่มในวิทยาศาสตร์ข้อมูล โน้ตบุ๊กนี้ครอบคลุมส่วนต่างๆ ดังนี้:

1.  [การจัดกลุ่ม (Grouping)](#Grouping)
    * [แยก, ประยุกต์, และรวม (Split, Apply, and Combine)](#Split,-Apply,-and-Combine)
    * [แบบฝึกหัดที่ 1 - อายุเฉลี่ยต่อมณฑล (Average Age Per County)](#Exercise-#1---Average-Age-Per-County)
2.  [การแบ่งกลุ่ม (Binning)](#Binning)
    * [แบบฝึกหัดที่ 2 - การใช้ Profiler (Using the Profiler)](#Exercise-#2---Using-the-Profiler)
3.  [การดำเนินการ Groupby ขั้นสูง (Advanced Groupby Operations)](#Advanced-Groupby-Operations)
    * [`.apply()`](#.apply())
    * [`.transform()`](#.transform())
4.  [ตาราง Pivot (Pivot Table)](#Pivot-Table)

## การจัดกลุ่ม (Grouping) ##

ในวิทยาศาสตร์ข้อมูล เรามักจะต้องการแบ่งข้อมูลออกเป็นกลุ่มและดำเนินการวิเคราะห์เพิ่มเติม เช่น:
* รวบรวมข้อมูลตามการจัดกลุ่ม
* เปรียบเทียบเมตริกซ์ในกลุ่มต่างๆ
* ทำความเข้าใจรูปแบบในข้อมูลทั่วทั้งกลุ่มต่างๆ
* ลบรายการที่ซ้ำกันหรือเติมค่าที่ขาดหายไปตามข้อมูลระดับกลุ่ม
* สร้างคุณลักษณะใหม่ตามสถิติระดับกลุ่ม
* รวมเข้ากับการแสดงข้อมูลด้วยภาพ

ด้านล่างนี้ เราจะโหลดชุดข้อมูลของเรา


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# DO NOT CHANGE THIS CELL
%load_ext cudf.pandas
import pandas as pd
import time

In [ ]:
# DO NOT CHANGE THIS CELL
dtype_dict={
    'age': 'int8', 
    'sex': 'category', 
    'county': 'category', 
    'lat': 'float32', 
    'long': 'float32', 
    'name': 'category'
}
        
df=pd.read_csv('./data/uk_pop.csv', dtype=dtype_dict)
df.head()

## แยก, ประยุกต์, และรวม (Split, Apply, and Combine) ##

เราใช้เมธอด `.groupby()` เพื่อจัดกลุ่มข้อมูลจำนวนมากและดำเนินการคำนวณกับกลุ่มเหล่านี้ การดำเนินการ groupby เกี่ยวข้องกับการรวมกันของการแยกวัตถุ การประยุกต์ใช้ฟังก์ชัน และการรวมผลลัพธ์ cuDF ใช้การจัดกลุ่มเรคคอร์ดในลักษณะที่เทียบเคียงได้กับ Pandas แต่มีความแตกต่างที่น่าสังเกตบางประการ

<p><img src='images/groupby.png' width=720></p>

cuDF รองรับการคำนวณและสถิติเชิงพรรณนา `DataFrameGroupBy` ทั่วไปหลายอย่าง เช่น `.size()`, `.mean()`, `.count()`, `.cov()`, `.cumprod()`, `.cumsum()`, `.max()`, `.min()`, `.nunique()`

**หมายเหตุ**: สามารถดูข้อมูลเพิ่มเติมเกี่ยวกับพฤติกรรมของ `.groupby()` สำหรับ pandas และความแตกต่างจาก cuDF ได้จากลิงก์ด้านล่าง:
* [pandas](https://pandas.pydata.org/docs/user_guide/groupby.html)
* [cuDF](https://docs.rapids.ai/api/cudf/stable/user_guide/groupby/)

ด้านล่างนี้ เราจะหาจำนวนคนในแต่ละมณฑล

In [ ]:
# DO NOT CHANGE THIS CELL
df.groupby('county').size()

**หมายเหตุ**: ผลลัพธ์ไม่ได้เรียงลำดับ เราสามารถเรียงลำดับผลลัพธ์ได้โดยใช้เมธอด `.sort_index()` หรือ `.sort_values()`

ด้านล่างนี้ เราจะนับจำนวนคนที่มีชื่อที่ได้รับความนิยมมากที่สุดและน้อยที่สุด


In [ ]:
# DO NOT CHANGE THIS CELL
df.groupby('name').size().sort_values()

ด้านล่างนี้ เราจะหาจุดศูนย์กลางโดยประมาณของแต่ละมณฑลโดยใช้ `.groupby().mean()` ในการดำเนินการ groupby เราควรจะ **รวมเฉพาะ** คอลัมน์ที่ถูกใช้งานเท่านั้น


In [ ]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL

county_center_df=df[['county', 'lat', 'long']].groupby('county')[['lat', 'long']].mean()
display(county_center_df)

In [ ]:
# DO NOT CHANGE THIS CELL
county_center_df.columns=['lat_county_center', 'long_county_center']
county_center_df.to_csv('county_centroid.csv')

### แบบฝึกหัดที่ 1 - อายุเฉลี่ยต่อมณฑล (Average Age Per County) ###

เราต้องการหาอายุเฉลี่ยสำหรับแต่ละมณฑล เราจะต้องใช้ทั้ง `.groupby()` และ `.sort_values()` โดยใช้เมธอด `.mean()` กับข้อมูลที่จัดกลุ่มตาม `county` ระบุมณฑล 5 อันดับแรกที่มีอายุเฉลี่ยสูงสุด

**คำแนะนำ**:
* แก้ไขเฉพาะส่วน `<FIXME>` เท่านั้น และรันเซลล์ด้านล่างเพื่อหาอายุเฉลี่ยสำหรับแต่ละมณฑล


In [ ]:
df[['county', 'age']].groupby(<<<<FIXME>>>>)['age']\
                     .<<<<FIXME>>>>()\
                     .sort_values(ascending=False)\
                     .head()

Click ... for solution. 

## การแบ่งกลุ่ม (Binning) ##

เมื่อจัดกลุ่มข้อมูลตัวเลขต่อเนื่อง บางครั้งการจัดกลุ่มตัวเลขให้อยู่ในช่วงหรือถังเก็บข้อมูลแบบไม่ต่อเนื่องก็มีประโยชน์ มีวิธีการแบ่งกลุ่มหลักๆ สองวิธี:

* การแบ่งกลุ่มแบบเท่ากัน: แบ่งช่วงออกเป็นช่วงขนาดเท่ากัน
* การแบ่งกลุ่มแบบกำหนดเอง: กำหนดถังเก็บข้อมูลแบบกำหนดเองตามความรู้ในโดเมนหรือเกณฑ์เฉพาะ

ฟังก์ชัน `.cut()` สามารถใช้เพื่อจัดกลุ่มค่าให้อยู่ในช่วงที่ไม่ต่อเนื่อง

In [ ]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL

bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
df['age_bucket']=pd.cut(df['age'].values, bins=bins, right=True, include_lowest=True, labels=False)
display(df.groupby('age_bucket').size())

### แบบฝึกหัดที่ 2 - การใช้ Profiler ###

cuDF pandas จะพยายามใช้ GPU เมื่อเป็นไปได้และจะกลับไปใช้ CPU สำหรับการดำเนินการบางอย่าง การรันโค้ดด้วยคำสั่ง magic command `cudf.pandas.line_profile` จะสร้างรายงานที่แสดงว่าการดำเนินการใดใช้ GPU และการดำเนินการใดใช้ CPU

**คำแนะนำ**:
* สังเกตว่าเซลล์ด้านล่างนี้เป็นการดำเนินการที่คล้ายกับเมื่อก่อนมาก ยกเว้นว่ามันใช้ฟังก์ชัน `range()` สำหรับพารามิเตอร์ `bins` ในสถานะปัจจุบัน สิ่งนี้ไม่ได้รับการสนับสนุนใน cuDF
* รันเซลล์ด้านล่างเพื่อรันการดำเนินการ binning บน CPU
* เปรียบเทียบเวลาที่ใช้ในการรันการดำเนินการที่คล้ายกันด้านบน

In [ ]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL

df['age_bucket']=pd.cut(df['age'].values, bins=range(0, 100, 10), right=True, include_lowest=True, labels=False)
display(df.groupby('age_bucket').size())

**หมายเหตุ**: ตัวโปรไฟล์สามารถช่วยเราระบุส่วนของโค้ดที่สามารถเขียนใหม่ให้เหมาะกับ GPU ได้มากขึ้น

## การดำเนินการ Groupby ขั้นสูง ##

เรายังสามารถใช้ตัวช่วยการประยุกต์ใช้ฟังก์ชันบนอินสแตนซ์ `DataFrameGroupBy` ได้ดังนี้:

* `DataFrameGroupby.aggregate()` / `Groupby.agg()` (นามแฝง): ใช้เมื่อเรามีการคำนวณเฉพาะสำหรับคอลัมน์ที่แตกต่างกัน หรือมีการคำนวณมากกว่าหนึ่งรายการบนคอลัมน์เดียวกัน
* `DataFrameGroupby.apply()`: ใช้เมื่อเราต้องการเรียกใช้ฟังก์ชันที่ผู้ใช้กำหนด (user-defined function) เฉพาะสำหรับแต่ละกลุ่ม
* `DataFrameGroupby.transform()`: ใช้เมื่อค่าผลลัพธ์ควรถูกกระจายไปทั่วทั้งกลุ่มและส่งคืน DataFrame ที่มีดัชนีเดียวกัน


### `.apply()` ###

เมธอด `.apply()` จะประยุกต์ใช้ฟังก์ชันตามกลุ่ม **ตามลำดับ** และรวมผลลัพธ์เข้าด้วยกัน เราสามารถส่งฟังก์ชัน callable เพื่อดำเนินการกับ DataFrame ทั้งหมดสำหรับแต่ละกลุ่มได้

ด้านล่างนี้ เราจะคำนวณระยะห่างของแต่ละบุคคลจากจุดศูนย์กลางของมณฑลของตน

In [ ]:
# DO NOT CHANGE THIS CELL

# define distance function
def distance(lat_1, long_1, lat_2, long_2): 
    return ((lat_2-lat_1)**2+(long_2-long_1)**2)**0.5

In [ ]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL

distance_df=df.groupby('county')[['lat', 'long']].apply(lambda x: distance(x['lat'], x['long'], x['lat'].mean(), x['long'].mean()))
df['R_1']=distance_df.reset_index(level=0, drop=True)

เราสามารถกำหนดฟังก์ชันแบบอินไลน์ได้ด้วย

In [ ]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL

df['R_2']=df.groupby('county')[['lat', 'long']].apply(lambda x: ((x['lat'].mean()-x['lat'])**2+(x['long'].mean()-x['long'])**2)**0.5).reset_index(level=0, drop=True)

**หมายเหตุ**: การดำเนินการนี้ค่อนข้างช้าเนื่องจากลักษณะการทำงานแบบวนซ้ำของเมธอด `.apply()`


### `.transform()` ###

เมธอด `.transform()` จะทำการรวมข้อมูลแต่ละกลุ่ม และกระจายผลลัพธ์ไปยังขนาดของกลุ่ม ทำให้ได้ DataFrame ที่มีขนาดและดัชนีเท่ากับ DataFrame ต้นฉบับ ภายใต้การทำงานภายใน เมธอด `.transform()` จะส่งแต่ละคอลัมน์ทีละคอลัมน์เป็น Series ไปยังฟังก์ชัน

ด้านล่างนี้ เราจะจัดกลุ่ม DataFrame ตาม `county` และแปลงคอลัมน์ `lat` และ `long` โดยใช้ `mean` เราจะลบค่าเฉลี่ยที่แปลงแล้วออกจากคอลัมน์ต้นฉบับ จากนั้นใช้สูตรระยะทางเพื่อคำนวณระยะทางที่ได้ โดยการรักษารูปร่างของ DataFrame ให้เหมือนเดิม เราสามารถดำเนินการ cuDF ได้อย่างรวดเร็ว ส่งผลให้ประสิทธิภาพเพิ่มขึ้น

In [ ]:
# DO NOT CHANGE THIS CELL
# make data types more precise
df[['lat', 'long']]=df[['lat', 'long']].astype('float64')

In [ ]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL

c=['lat', 'long']
df['R_3']=((df[c] - df.groupby('county')[c].transform('mean')) ** 2).sum(axis=1) ** 0.5

In [ ]:
df.head()


แม้ว่าเมธอด `.apply()` จะมีความยืดหยุ่นมากกว่าและสามารถจัดการการดำเนินการที่ซับซ้อนได้ แต่โดยทั่วไปแล้วจะช้ากว่า ในทางกลับกัน เมธอด `.transform()` สามารถทำงานได้เร็วกว่ามาก เมื่อเราออกแบบขั้นตอนเพื่อใช้การดำเนินการแบบเวกเตอร์ เราจะได้รับประโยชน์ด้านประสิทธิภาพอย่างมาก

**หมายเหตุ**: `Groupby.apply()` ไม่ได้ปรับขนาดได้ดีกับจำนวนกลุ่ม ดังนั้นความแตกต่างของประสิทธิภาพนี้จะเห็นได้ชัดเจนยิ่งขึ้นเมื่อมีจำนวนกลุ่มที่สูงขึ้น


## ตาราง Pivot (Pivot Table) ##

ตาราง Pivot ช่วยให้เราสามารถสรุปและรวมชุดข้อมูลขนาดใหญ่ให้อยู่ในรูปแบบที่จัดการได้ง่ายขึ้นสำหรับการวิเคราะห์ เมื่อใช้ `DataFrame.pivot_table()` เราจะระบุอาร์กิวเมนต์ `index`, `columns` และ `values` รวมถึง `aggfunc` สิ่งนี้จะจัดกลุ่มข้อมูลตาม `index` และ `columns` และดำเนินการรวมข้อมูลบน `values` เราสามารถใช้ฟังก์ชันการรวมข้อมูลหลายฟังก์ชัน ซึ่งโดยทั่วไปจะเร็วกว่าและประหยัดหน่วยความจำมากกว่าการจัดกลุ่มและรวมข้อมูลด้วยตนเองสำหรับชุดข้อมูลขนาดใหญ่

ด้านล่างนี้ เราจะสร้างตาราง Pivot ที่นับจำนวนเพศแต่ละเพศในแต่ละมณฑล นอกจากนี้ เราจะหาเปอร์เซ็นต์ของจำนวนทั้งหมดสำหรับแต่ละมณฑล

In [ ]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL

pvt_tbl=df[['county', 'sex', 'name']].pivot_table(index=['county'], columns=['sex'], values='name', aggfunc='count')
pvt_tbl=pvt_tbl.apply(lambda x: x/sum(x), axis=1)
display(pvt_tbl)

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**เยี่ยมมาก!** ไปยัง [โน้ตบุ๊กถัดไป](1-06_data_visualization.ipynb) กันเลย